In [5]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
              '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']
from plotly.offline import init_notebook_mode, iplot

In [6]:
import numpy as np 
import pandas as pd
import json

import folium
import geopandas as gpd
from shapely.geometry import Point
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import json
import os

import geopandas as gpd
from geopandas import GeoDataFrame
pd.set_option('display.max_columns', None)  


In [7]:
df = pd.read_csv("./hackathon/hackathon/data/train.csv")
df.head()

In [8]:
map_ = folium.Map(location=[12.93447,77.634470], tiles='cartodbpositron', zoom_start=5)

for idx, row in df.head(200).iterrows():
    Marker([row['latitude'], row['longitude']]).add_to(map_)

map_

In [9]:
# df[['latitude', 'longitude', 'rent']].describe()
dict_columns = pd.DataFrame([(json.loads(df.amenities.loc[i])) for i in range(len(df))])
dict_columns

In [10]:
dict_columns.isna().sum()

In [11]:
dict_columns = dict_columns.replace({True: 1, False: 0})

In [159]:
test_dict

In [12]:
df.drop(["amenities"], inplace=True, axis=1)

In [13]:
df = pd.concat([df, dict_columns], axis=1)

In [14]:
df.describe()

In [15]:
obj_names=["type","lease_type","furnishing","parking","facing","water_supply","building_type"]
obt_type = df[obj_names]
obt_type

In [16]:
from sklearn.preprocessing import OrdinalEncoder
# encode1 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encode1 = OrdinalEncoder()
encoder = encode1.fit(obt_type)
encoded = encoder.transform(obt_type)
encoded

In [17]:
df[["type","lease_type","furnishing","parking","facing","water_supply","building_type"]] = encoded
df

In [19]:
df["year"] = pd.to_datetime(df['activation_date'] , format="%d-%m-%Y %H:%M").dt.year
df["month"] = pd.to_datetime(df['activation_date'] , format="%d-%m-%Y %H:%M").dt.month

####  **There are some unique values in it which might be useful**

#### **Let's just drop Locality column for now**

In [20]:
locality_var = df['locality']
df.drop(["locality", "GYM"], axis=1, inplace=True)

In [21]:
df.isnull().sum()

In [22]:
cate_names=["CLUB","CPA","SERVANT","GP","RWH","STP","VP"]
cate_type = df[cate_names]
cate_type.head()

In [23]:
from sklearn.impute import SimpleImputer
imp_freq = SimpleImputer(strategy='most_frequent')

imp_freq_cate = imp_freq.fit(cate_type)
transformed_cate_type = imp_freq_cate.transform(cate_type)
transformed_cate_type

In [24]:
df[cate_names] = transformed_cate_type
df.head()

**Let's drop activation_date for now to build a model and see how it works**

In [25]:
actication_date = df.activation_date
df.drop(["activation_date","id"],axis=1, inplace=True)
df.drop(["LIFT","POOL"],axis=1, inplace=True)

In [26]:
df["bathroom"].value_counts()

In [27]:
df.describe()

In [28]:
df.head()

In [29]:
cat_columns = ['type', 'lease_type', 'gym', 'lift',
       'swimming_pool', 'negotiable', 'furnishing', 'parking',
        'bathroom', 'facing', 'floor',
       'total_floor', 'water_supply', 'building_type', 'balconies',
       'INTERNET', 'AC', 'CLUB', 'INTERCOM', 'CPA', 'FS', 'SERVANT',
       'SECURITY', 'SC', 'GP', 'PARK', 'RWH', 'STP', 'HK', 'PB', 'VP', 'year',
       'month']

In [30]:
def cat_summary_with_graph(dataframe, col_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentages'),
                        specs=[[{"type": "xy"}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=dataframe[col_name].value_counts().values.tolist(),
                         x=[str(i) for i in dataframe[col_name].value_counts().index],
                         text=dataframe[col_name].value_counts().values.tolist(),
                         textfont=dict(size=15),
                         name=col_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=colors,
                                     line=dict(color='#DBE6EC',
                                               width=1))),
                  row=1, col=1)

    fig.add_trace(go.Pie(labels=dataframe[col_name].value_counts().keys(),
                         values=dataframe[col_name].value_counts().values,
                         textfont=dict(size=20),
                         textposition='auto',
                         showlegend=False,
                         name=col_name,
                         marker=dict(colors=colors)),
                  row=1, col=2)

    fig.update_layout(title={'text': col_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')

    iplot(fig)


In [31]:
for col in cat_columns:
    cat_summary_with_graph(df,col)

In [32]:
df.head()

In [33]:
import seaborn as sns
ax = sns.regplot(x="property_size", y="rent", data=df)

In [34]:
ax = sns.regplot(x="property_age", y="rent", data=df)

In [35]:
import matplotlib.pyplot as plt
sns.set_style('ticks')
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(40, 45)
sns.heatmap(data=df.corr(),cmap='Blues',annot=True)

In [36]:
X = df.drop('rent',axis=1)
y = df['rent']

In [37]:
from sklearn.linear_model import LinearRegression, BayesianRidge, HuberRegressor
from sklearn.neural_network import BernoulliRBM
from sklearn.svm import LinearSVR,SVR,NuSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,auc
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.9, random_state=0)

### **Random Forest**

In [123]:
param_grid_rf = {
               'n_estimators': [int(x) for x in np.linspace(5, 200, num = 5)],
               'min_samples_split': [int(x) for x in np.linspace(1, 50, num =5)],
               'min_samples_leaf': [int(x) for x in np.linspace(1, 20, num = 5)],
                'max_features' : ["sqrt", None],
                'bootstrap':[True]
              }


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor


In [55]:

rf = RandomForestRegressor()

grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid_rf, 
                          cv = 5, n_jobs = -1, verbose = 2)

rf_model = grid_search_rf.fit(X_train, y_train)

In [57]:
mean_squared_error(y_val, rf_model.predict(X_val), squared=False)

In [40]:
van_rf = AdaBoostRegressor(RandomForestRegressor(n_estimators = 151, min_samples_split = 13, min_samples_leaf= 1, max_features = None, bootstrap=True) )
van_rf.fit( X_train, y_train) 
mean_squared_error(y_val, van_rf.predict(X_val), squared=False)  #4128

### **Decision Tree**

In [122]:
max_features = ['auto', 'sqrt', "log2"]

max_depth = [int(x) for x in np.linspace(5, 400, num = 40)]

min_samples_split =[int(x) for x in np.linspace(1, 50, num =10)]

min_samples_leaf = [int(x) for x in np.linspace(1, 20, num = 10)]

# Create the random grid
param_grid_dt = {
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
              }



In [43]:
dt = DecisionTreeRegressor()

grid_search_dt = GridSearchCV(estimator = dt, param_grid = param_grid_dt, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [44]:
dt_model = grid_search_dt.fit(X_train, y_train)

In [45]:
mean_squared_error(y_val, dt_model.predict(X_val), squared=False)

In [46]:
best_grid_dt = grid_search_dt.best_estimator_
best_grid_dt

In [47]:
van_dt = DecisionTreeRegressor(max_depth=15,
 max_features= 'auto',
 min_samples_leaf= 11,
 min_samples_split=33).fit( X_train, y_train) 

mean_squared_error(y_val, van_dt.predict(X_val), squared=False)

In [48]:

ada_dt = AdaBoostRegressor(DecisionTreeRegressor(
    max_depth=15,
    max_features= 'auto',
    min_samples_leaf= 11,
    min_samples_split=33
     ))
ada_dt.fit( X_train, y_train) 

In [49]:
mean_squared_error(y_val, ada_dt.predict(X_val), squared=False)

### XG boost

In [89]:
import xgboost as xgb

In [124]:
regressor=xgb.XGBRegressor(learning_rate = 0.02,
                           n_estimators  = 733,
                           max_depth     = 5,
                           eval_metric='rmse')

regressor.fit(X_train, y_train)

In [96]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=20, ax=ax)
plt.show();

In [127]:
features=['longitude', 'latitude', 'property_size', 'type', 'property_age',
       'cup_board', 'total_floor', 'month','floor', 'lease_type', 'facing', 'furnishing', 'bathroom', 'balconies', 'parking', 'GP','building_type', 'water_supply', 'negotiable', 'lift']

In [186]:
y = df['rent']
temp_x = X[features]
New_X_train, New_X_val, New_y_train, New_y_val = train_test_split(temp_x,y, test_size=0.9, random_state=0)

In [109]:
import xgboost as xgb


regressor=xgb.XGBRegressor(eval_metric='rmse')

from sklearn.model_selection import GridSearchCV

param_grid1 = {"max_depth":    [int(x) for x in np.linspace(5, 20, num = 4)],
              "n_estimators": [int(x) for x in np.linspace(200, 800, num = 10)],
              "learning_rate": [0.01, 0.02],}

search = GridSearchCV(regressor, param_grid1, cv=5,verbose=2).fit(New_X_train, New_y_train)

In [145]:
xgb_model = xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],
#                            max_leaves = search.best_params_["max_leaves"],
                           eval_metric='rmse')

xgb_model.fit(New_X_train, New_y_train)

In [146]:
mean_squared_error(New_y_val, xgb_model.predict(New_X_val), squared=False)

In [142]:
regressor=AdaBoostRegressor(xgb.XGBRegressor(learning_rate = 0.02,
                           n_estimators  = 733,
                           max_depth     = 5,
                           eval_metric='rmse'))

regressor.fit(New_X_train, New_y_train)

In [184]:
mean_squared_error(New_y_val, xgb_model.predict(New_X_val), squared=False)

# Submission

In [185]:
test=pd.read_csv("./hackathon/hackathon/data/test.csv")
test_dict=pd.DataFrame([(json.loads(test.amenities.loc[i])) for i in range(len(test))])

test_dict =  test_dict.replace({True: 1, False: 0})
test.drop(["amenities"], inplace=True, axis=1)
test = pd.concat([test, test_dict], axis=1)

test_obj_type = test[obj_names]

encoded_test = encoder.transform(test_obj_type)
test[["type","lease_type","furnishing","parking","facing","water_supply","building_type"]] = encoded_test


test["year"] = pd.to_datetime(test['activation_date'] , format="%d-%m-%Y %H:%M").dt.year
test["month"] = pd.to_datetime(test['activation_date'] , format="%d-%m-%Y %H:%M").dt.month

test.drop(["locality", "GYM"], axis=1, inplace=True)

cate_test_type=test[cate_names]

transformed_test_cate_type =  imp_freq_cate.transform(cate_test_type)
test[cate_names] = transformed_test_cate_type

test.drop(["activation_date","id"],axis=1, inplace=True)
test.drop(["LIFT","POOL"],axis=1, inplace=True)

In [155]:
import pickle
pickle.dump(regressor, open('model.sav', 'wb'))

In [172]:
pred = regressor.predict(test[features])
pred

In [183]:
pd.DataFrame({"rent":pred}).to_csv("submission_final.csv",index=False)